## 第 3 章 KNN 算法

K 近邻算法（K-Nearest Neighbors，KNN）是最经典、最简单的一类监督学习方法，可做**分类**也可做**回归**。

核心想法：

> 新样本长得像谁，就按谁来分类/预测。

具体做法：算新样本到所有训练样本的**距离**，找出最近的 K 个“邻居”，再根据这些邻居的标签来决定新样本的标签或数值。


### 3.1 KNN 算法介绍

#### 3.1.1 工作原理

1. **计算距离**
   对每一个待预测样本 (x)，计算它和训练集中每个样本 (x^{(i)}) 的距离。

2. **选择 K 个近邻**
   按距离从小到大排序，选出最近的 K 个样本。

3. **投票或平均**

   * **分类任务**：看这 K 个近邻中，哪个类别出现得最多，就预测为哪个类别（多数投票）。
   * **回归任务**：取这 K 个近邻标签值的平均数，作为预测结果。

总结：

> KNN 不“学参数”，而是“记住数据”，预测时再来“查邻居”。

#### 3.1.2 关键参数

1. **距离度量方法**

常见的几种距离（具体公式在 3.2 小节）：

* 欧氏距离（Euclidean）
* 曼哈顿距离（Manhattan）
* 切比雪夫距离（Chebyshev）
* 闵可夫斯基距离（Minkowski）

在 `sklearn` 中，KNN 默认使用欧氏距离。

2. **K 值（邻居个数）**

* K 太小：模型对训练数据太敏感，**容易过拟合**（对噪声很敏感）。
* K 太大：把很远的点也考虑进来，**容易欠拟合**（过于“平均”）。

一般会通过**交叉验证 + 网格搜索**来选 K（见 3.5）。

#### 3.1.3 优缺点

**优点**

* 思路简单、非常直观；
* 模型本身几乎没有训练过程（只是存数据），实现方便。

**缺点**

* 预测时需要和所有训练样本算距离，**计算量大**，数据多时会变慢；
* 对**特征尺度**和**噪声数据**敏感（所以常配合归一化/标准化使用）。

**适用场景**：
数据量不是特别大、特征数适中，且对解释性要求不高的分类/回归问题。

#### 3.1.4 KNN 的 API 使用

下面用 `sklearn` 演示 KNN 的分类和回归。

##### 1）KNN 分类

In [1]:
from sklearn.neighbors import KNeighborsClassifier

# 1. 构建模型，K=2
knn = KNeighborsClassifier(n_neighbors=2)

# 2. 准备数据
X = [[2, 1], [3, 1], [1, 4], [2, 6]]  # 特征
y = [0, 0, 1, 1]                      # 标签

# 3. 训练模型
knn.fit(X, y)

# 4. 预测新样本
y_pred = knn.predict([[4, 9]])
print(y_pred)


[1]


**结果说明**：
输出是预测的类别（例如 `[1]`），表示新样本被分到类别 1。

**应用场景**：
根据多个特征（比如身高、体重、年龄等）来判断一个人属于哪一类（如是否患病、是否流失等）。

##### 2）KNN 回归

In [2]:
from sklearn.neighbors import KNeighborsRegressor

# 1. 构建模型，K=2
knn = KNeighborsRegressor(n_neighbors=2)

# 2. 准备数据
X = [[2, 1], [3, 1], [1, 4], [2, 6]]  # 特征
y = [0.5, 0.33, 4, 3]                 # 连续值标签

# 3. 训练模型
knn.fit(X, y)

# 4. 预测新样本
y_pred = knn.predict([[4, 9]])
print(y_pred)

[3.5]


**结果说明**：
输出是一个连续值（例如 `[某个小数]`），是 K 个近邻标签的平均值。

**应用场景**：
房价预测、评分预测等连续数值问题。

### 3.2 常见距离度量方法（了解）

设有两个点
$(x = (x_1, x_2, \dots, x_n))$，
$(y = (y_1, y_2, \dots, y_n))$。

#### 3.2.1 欧氏距离（Euclidean Distance）

欧氏距离就是“**直线距离**”，可以理解为尺子量的那条线的长度。

$$
d(x, y) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}
$$

**应用**：

适合连续数值特征，常用默认距离。

#### 3.2.2 曼哈顿距离（Manhattan Distance）

曼哈顿距离是两点在坐标轴方向上的“**绝对轴距之和**”。

$$
d(x, y) = \sum_{i=1}^n \lvert x_i - y_i \rvert
$$

为什么叫“曼哈顿”距离？
可以想象在曼哈顿这种网格状街区，车不能斜着走，只能沿“横竖街道”走，所以实际距离是“**横着走多少 + 竖着走多少**”。

**应用**：

对“走格子”的路径更自然，某些高维场景下比欧氏距离更稳定。

#### 3.2.3 切比雪夫距离（Chebyshev Distance）

切比雪夫距离是两点各坐标差值的**最大值**：

$$
d(x, y) = \max_i \lvert x_i - y_i \rvert
$$

**例子：国际象棋国王**
国王可以横、竖、斜走一步，它从起点到终点最少需要走的步数，就等于这两点的切比雪夫距离。

**应用**：

更看重“**差得最多的那一维**”。

#### 3.2.4 闵可夫斯基距离（Minkowski Distance）

闵可夫斯基距离是一个“**通用版距离公式**”，通过一个参数 (p) 控制距离的形态：

$$
d(x, y) = \left( \sum_{i=1}^n \lvert x_i - y_i \rvert^p \right)^{1/p}
$$

当 (p) 取不同值时，可以变成不同的具体距离：

* $(p = 1)$：曼哈顿距离
  $$
  d(x, y) = \sum_{i=1}^n \lvert x_i - y_i \rvert
  $$
* $(p = 2)$：欧氏距离
  $$
  d(x, y) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2}
  $$
* $(p \to \infty)$：切比雪夫距离
  $$
  d(x, y) = \max_i \lvert x_i - y_i \rvert
  $$

**应用**：

通过调节 (p)，你可以在“更关注整体差异”（小 (p)）和“更关注最大差异”（大 (p)）之间切换。

### 3.3 归一化与标准化

KNN 是基于“**距离**”的算法，所以不同特征的**量纲和尺度**会对结果影响很大。比如：

* 身高 1.7，体重 70
* 如果不处理，体重的数值要远大于身高，对距离影响会更大。

因此，常用两种方法：**归一化**和**标准化**。

#### 3.3.1 归一化（Normalization）

##### 1）定义

把数据线性缩放到一个固定区间 ($[x_{\min}, x_{\max}]$)，常用的是 ($[0, 1]$) 或 ($[-1, 1]$)。

* 映射到 ($[0, 1]$)：
  $$
  x' = \frac{x - x_{\min}}{x_{\max} - x_{\min}}
  $$

* 映射到 ($[-1, 1]$)：
  $$
  x' = 2 \times \frac{x - x_{\min}}{x_{\max} - x_{\min}} - 1
  $$

##### 2）作用

* **消除量纲差异**：身高（米）、体重（千克）等不同单位不再“抢权重”；
* **加速模型收敛**：对用梯度下降优化的模型（如神经网络）帮助更大；
* **适配对范围敏感的模型**：如 KNN、SVM、神经网络等。

##### 3）适用场景

* 数据有明显边界：如图像像素（0–255）、概率、频率等；
* 模型对特征范围敏感时优先考虑归一化；
* 对异常值比较敏感：极端值会把区间拉得很大。

##### 4）`sklearn` API 使用

In [3]:
from sklearn.preprocessing import MinMaxScaler

X = [[2, 1], [3, 1], [1, 4], [2, 6]]

# 归一化，区间设置为 (-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

print(X_scaled)

[[ 0.  -1. ]
 [ 1.  -1. ]
 [-1.   0.2]
 [ 0.   1. ]]


**结果说明**：
`X_scaled` 中每一列特征都被压缩到了 `(-1, 1)` 范围内。

**应用场景**：
常和 KNN、神经网络一起使用，避免某个大数值特征“主导”距离。

#### 3.3.2 标准化（Standardization）

##### 1）定义

把数据变换为**均值为 0、标准差为 1** 的分布：

$$
x' = \frac{x - \mu}{\sigma}
$$

其中：

* 均值：
  $$
  \mu = \frac{1}{n} \sum_{i=1}^n x_i
  $$
* 标准差：
  $$
  \sigma = \sqrt{\frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2}
  $$

##### 2）作用

* **适配假设正态分布的模型**：如线性回归、逻辑回归等；
* 对**异常值不如归一化那么敏感**，模型更稳定；
* 同样能消除不同特征的量纲差异。

##### 3）适用场景

* 数据分布未知或接近正态分布；
* 存在少量异常值时，比简单归一化更稳；
* 非距离型模型也经常使用（如线性模型、SVM 等）。

##### 4）`sklearn` API 使用

In [4]:
from sklearn.preprocessing import StandardScaler

X = [[2, 1], [3, 1], [1, 4], [2, 6]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X_scaled)

[[ 0.         -0.94280904]
 [ 1.41421356 -0.94280904]
 [-1.41421356  0.47140452]
 [ 0.          1.41421356]]


**结果说明**：
`X_scaled` 中每一列特征大致满足“均值约为 0，标准差约为 1”。

**应用场景**：
多数情况下，标准化比归一化更“通用”，尤其在不确定数据分布时。

### 3.4 案例：心脏病预测

这一节用 KNN 做一个实际任务：**预测是否患有心脏病**。

#### 3.4.1 数据集说明

使用 Kaggle 上的 [Heart Disease Dataset](https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset)。

主要特征（列）示例（中文命名仅用于说明）：

* 年龄：连续值
* 性别：0-女，1-男
* 胸痛类型：0-典型心绞痛，1-非典型心绞痛，2-非心绞痛，3-无症状
* 静息血压：连续值，单位 mmHg
* 胆固醇：连续值，单位 mg/dl
* 空腹血糖：1-大于 120 mg/dl，0-小于等于 120 mg/dl
* 静息心电图结果：0-正常，1-ST-T 异常，2-可能左心室肥大
* 最大心率：连续值
* 运动性心绞痛：1-有，0-无
* 运动后的 ST 下降：连续值
* 峰值 ST 段的斜率：0-向上，1-水平，2-向下
* 主血管数量：0 到 3
* 地中海贫血：0-正常，1-固定缺陷，2-可逆缺陷
* 是否患有心脏病：**标签**，0-否，1-是

#### 3.4.2 加载数据集

In [5]:
import pandas as pd

# 加载数据集
heart_disease = pd.read_csv("data/heart_disease.csv")

# 处理缺失值（这里简单地丢弃有缺失的行）
heart_disease = heart_disease.dropna()

# 查看数据概况
heart_disease.info()
print(heart_disease.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   年龄        1025 non-null   int64  
 1   性别        1025 non-null   int64  
 2   胸痛类型      1025 non-null   int64  
 3   静息血压      1025 non-null   int64  
 4   胆固醇       1025 non-null   int64  
 5   空腹血糖      1025 non-null   int64  
 6   静息心电图结果   1025 non-null   int64  
 7   最大心率      1025 non-null   int64  
 8   运动性心绞痛    1025 non-null   int64  
 9   运动后的ST下降  1025 non-null   float64
 10  峰值ST段的斜率  1025 non-null   int64  
 11  主血管数量     1025 non-null   int64  
 12  地中海贫血     1025 non-null   int64  
 13  是否患有心脏病   1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB
   年龄  性别  胸痛类型  静息血压  胆固醇  空腹血糖  静息心电图结果  最大心率  运动性心绞痛  运动后的ST下降  峰值ST段的斜率  \
0  52   1     0   125  212     0        1   168       0       1.0         2   
1  53   1     0   140  203     1        0   155       1   

**结果说明**：

* `info()` 查看字段名、非空数量、数据类型；
* `head()` 看前几行数据，确认字段是否正确读取。

#### 3.4.3 数据集划分

In [6]:
from sklearn.model_selection import train_test_split

# 特征和标签
X = heart_disease.drop("是否患有心脏病", axis=1)  # 特征
y = heart_disease["是否患有心脏病"]             # 标签

# 按 7:3 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=100
)

**应用场景**：
通过划分训练集/测试集，可以在训练后用**没见过的数据**评估模型泛化能力。

#### 3.4.4 特征工程

KNN 基于距离，所以我们要特别注意**特征类型**和**尺度**。

##### 1）特征类型与预处理

* **类别型特征（需要独热编码）**

  * 胸痛类型
  * 静息心电图结果
  * 峰值 ST 段的斜率
  * 地中海贫血

  这些特征如果用整数编码（0, 1, 2, 3），KNN 会把它们当成“有大小关系”的数值，用距离来衡量差异，这是错误的。因此，需要用**独热编码（One-Hot Encoding）**打散成多个 0/1 特征，消除伪“顺序”。

* **数值型特征（可标准化）**

  * 年龄、静息血压、胆固醇、最大心率、运动后的 ST 下降、主血管数量

* **二元特征（可直接保留）**

  * 性别、空腹血糖、运动性心绞痛

使用 `ColumnTransformer` 统一处理：

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 数值型特征
numerical_features = ["年龄", "静息血压", "胆固醇", "最大心率", "运动后的ST下降", "主血管数量"]

# 类别型特征
categorical_features = ["胸痛类型", "静息心电图结果", "峰值ST段的斜率", "地中海贫血"]

# 二元特征
binary_features = ["性别", "空腹血糖", "运动性心绞痛"]

# 列转换器：不同列使用不同的预处理方式
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),               # 数值特征标准化
        ("cat", OneHotEncoder(drop="first"), categorical_features),  # 类别特征独热编码
        ("binary", "passthrough", binary_features),                  # 二元特征原样保留
    ]
)

# 对训练集和测试集做相同转换（注意：fit 只在训练集上做）
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

**结果说明**：

* 数值特征被标准化为“均值 0、方差 1”；
* 类别特征变成多个 0/1 列；
* 二元特征原样保留；
* 组合起来得到一个适合 KNN 使用的特征矩阵。

##### 2）为什么 `drop="first"` 可以避免多重共线性？

以特征“胸痛类型”为例，包含 4 个类别（0, 1, 2, 3）。
如果直接做独热编码，会生成 4 列：

* 胸痛类型_0
* 胸痛类型_1
* 胸痛类型_2
* 胸痛类型_3

对于任一样本，这 4 列中**恰好有一列为 1，其余为 0**，所以总是满足：

$$
\text{胸痛类型}_0 + \text{胸痛类型}_1 + \text{胸痛类型}_2 + \text{胸痛类型}_3 = 1
$$

这 4 列存在完全线性相关关系，称为**多重共线性（Multicollinearity）**，会导致有些模型（尤其是线性模型）在求参数时矩阵不可逆或接近奇异，参数估计不稳定。

设置 `drop="first"` 后，会删掉每个类别特征的一列，例如只保留：

* 胸痛类型_1
* 胸痛类型_2
* 胸痛类型_3

此时：

* 当三列全为 0 时，隐含表示“胸痛类型 = 0”；
* 这样就打破了完全线性关系，同时减少一列冗余特征。

> 虽然 KNN 不是线性模型，对多重共线性不敏感，但减少冗余特征可以**减少计算量**，提高效率。

#### 3.4.5 模型训练与评估

In [8]:
from sklearn.neighbors import KNeighborsClassifier

# 使用 K 近邻分类模型，K=3
knn = KNeighborsClassifier(n_neighbors=3)

# 训练模型
knn.fit(x_train, y_train)

# 在测试集上评估模型准确率
score = knn.score(x_test, y_test)
print("测试集准确率：", score)

测试集准确率： 0.9188311688311688


**结果说明**：
`score` 是在测试集上的分类准确率，例如 `0.85` 表示 85% 的样本预测正确。

**应用场景**：
快速得到一个基准模型，看这个任务用 KNN 能达到什么水平。

#### 3.4.6 模型的保存与加载

用 `joblib` 保存训练好的模型，方便以后直接加载使用。

In [9]:
import joblib

# 保存模型
joblib.dump(knn, "knn_heart_disease.joblib")

['knn_heart_disease.joblib']

加载模型并预测：

In [10]:
import joblib

# 加载模型
knn_loaded = joblib.load("knn_heart_disease.joblib")

# 预测第 11 个测试样本（索引 10）
y_pred = knn_loaded.predict(x_test[10:11])

# 打印真实值与预测值
print("真实值：", y_test.iloc[10])
print("预测值：", y_pred[0])

真实值： 0
预测值： 0


**应用场景**：
训练一次模型，保存下来，在实际系统中反复使用，不必每次都重新训练。

### 3.5 模型评估与超参数调优

#### 3.5.1 网格搜索（Grid Search）

网格搜索是一种**系统化的超参数调优方法**：

* 先设定一个“参数网格”，列出所有要尝试的参数组合；
* 然后对每一个参数组合做一次训练 + 评估；
* 选出表现最好的那一组参数。

通常会结合**交叉验证（Cross Validation）**一起用：

* 外层：遍历所有参数组合；
* 内层：对每个组合做 k 折交叉验证，得到一个平均性能指标。

好处：**自动化调参**，减少靠感觉试参数的过程。

#### 3.5.2 对心脏病预测模型进行超参数调优

下面对前面的 KNN 模型做超参数调优（主要调 K 值）：

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

# 基础模型（不指定 n_neighbors）
knn = KNeighborsClassifier()

# 网格搜索参数：K 从 1 到 10
param_grid = {"n_neighbors": list(range(1, 11))}

# GridSearchCV(estimator=模型, param_grid=参数网格, cv=交叉验证折数)
grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=10
)

# 使用训练集进行网格搜索 + 交叉验证
grid_search.fit(x_train, y_train)

# 所有交叉验证结果
print(pd.DataFrame(grid_search.cv_results_))

# 最佳模型、最佳得分
print("最佳模型：", grid_search.best_estimator_)
print("最佳交叉验证得分：", grid_search.best_score_)

# 使用最佳模型在测试集上进行评估
best_knn = grid_search.best_estimator_
test_score = best_knn.score(x_test, y_test)
print("测试集准确率（最佳 K）：", test_score)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.000572      0.000292         0.001267        0.000637   
1       0.000383      0.000134         0.000745        0.000254   
2       0.000370      0.000148         0.001088        0.000482   
3       0.000291      0.000035         0.001304        0.000433   
4       0.000364      0.000115         0.001210        0.000548   
5       0.000323      0.000103         0.000962        0.000514   
6       0.000320      0.000040         0.001161        0.000250   
7       0.000289      0.000040         0.000939        0.000321   
8       0.000293      0.000034         0.001021        0.000306   
9       0.000380      0.000104         0.001149        0.000257   

   param_n_neighbors               params  split0_test_score  \
0                  1   {'n_neighbors': 1}           0.986111   
1                  2   {'n_neighbors': 2}           0.930556   
2                  3   {'n_neighbors': 3}           0.888889   
3     

**结果说明**：

* `best_estimator_`：找到的最佳 KNN 模型（包括最佳的 `n_neighbors`）；
* `best_score_`：在交叉验证中的平均最佳表现；
* `test_score`：最佳模型在测试集上的最终表现。

**应用场景**：
当你不确定 K 取多少、或者还有其他参数（如距离度量等）需要调时，用网格搜索可以**系统地探索参数空间**，找到更好的模型配置。
